In [1]:
from GUI import GUI

In [4]:
gui = GUI(img_file='data/input/4.jpg')

In [6]:
gui.element_detection()
gui.visualize_element_detection()

*** Please replace the Google OCR key at detect_text/ocr.py line 28 with your own (apply in https://cloud.google.com/vision) ***
[Text Detection Completed in 3.567 s] data/input/4.jpg
(800, 467)
[Compo Detection Completed in 449.099 s] data/input/4.jpg
Merge Complete and Save to data/output\uied\4.jpg Wed Aug  4 15:47:30 2021 



In [4]:
gui.layout_recognition()
gui.visualize_layout_recognition()